<a href="https://colab.research.google.com/github/Amr789/FaceAnalysis-System/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Clone repository (or pull updates if it exists)
import os
if not os.path.exists("FaceAnalysis-System"):
    !git clone https://github.com/Amr789/FaceAnalysis-System.git
else:
    %cd FaceAnalysis-System
    !git pull

# 2. Enter the directory
if os.getcwd().split('/')[-1] != "FaceAnalysis-System":
    %cd FaceAnalysis-System

# 3. Install dependencies
print("Installing dependencies...")
!pip install -r requirements.txt -q

# 4. Mount Google Drive (To save your trained model)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Upload your kaggle.json when prompted
from google.colab import files
import os

if not os.path.exists("kaggle.json"):
    print("Please upload your kaggle.json file:")
    files.upload()

!python main.py --mode setup

In [ ]:
# Run training using the main entry point
!python main.py --mode train --epochs 15 --batch 32

In [ ]:
import matplotlib.pyplot as plt
import cv2
from src.inference import UTKFacePipeline
from google.colab import files

# Initialize pipeline (Loads EfficientNet weights)
pipeline = UTKFacePipeline(age_model_path="models/utk_age_model.pth")

# Upload images
print("Upload 2 images to compare:")
uploaded = files.upload()
paths = list(uploaded.keys())

if len(paths) >= 2:
    # 1. Run Inference
    result = pipeline.compare(paths[0], paths[1])

    # 2. Setup Visualization
    # Load images (OpenCV loads in BGR, so we convert to RGB for Matplotlib)
    img1 = cv2.cvtColor(cv2.imread(paths[0]), cv2.COLOR_BGR2RGB)
    img2 = cv2.cvtColor(cv2.imread(paths[1]), cv2.COLOR_BGR2RGB)

    # Create the plot
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))

    # --- Left Image ---
    ax[0].imshow(img1)
    ax[0].set_title(f"Predicted Age: {result['age_1']} years", fontsize=12)
    ax[0].axis('off')

    # --- Right Image ---
    ax[1].imshow(img2)
    ax[1].set_title(f"Predicted Age: {result['age_2']} years", fontsize=12)
    ax[1].axis('off')

    # --- Main Header (The Green/Red Result) ---
    match_text = "SAME PERSON" if result['match'] else "DIFFERENT PEOPLE"
    match_color = "green" if result['match'] else "red"

    plt.suptitle(f"Result: {match_text}\nSimilarity: {result['similarity']}",
                 fontsize=16, fontweight='bold', color=match_color)

    # Show the plot
    plt.tight_layout()
    plt.show()

    # 3. Print the Raw Data (Footer)
    print(f"📊 Full Data: {result}")
else:
    print("⚠️ Please upload at least 2 images.")